In [6]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random


import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.patches import Ellipse, Circle


# User defined Function

In [28]:
'''Time stamp'''
def stamp_transit(time_str):
    timeArray = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    return timeStamp
    
'''Time step'''

def stamp_to_step(timestamp,date_str,step):
    baseline = date_str+" 00:00:00";
    baseline = int(stamp_transit(baseline))
    current_step=int((timestamp-baseline)/step)
    return current_step

'''Grid Verifying'''

def IntersectsSegment(ptx,pty,sptx,spty,eptx,epty): #0 for no, 1 for yes
    #input: test point, start point , end point; all in format [lng, lat]
    
    if spty>pty and epty>pty: #segment over point
        return 0
    if spty<pty and epty<pty: #segment under point
        return 0
    if epty==pty: #end point is not counted
        return 0
    if sptx<ptx and eptx<ptx: #segment is on the left of test point
        return 0
    if spty==epty: #error
        print('error')
        return 0

    xseg=eptx-(eptx-sptx)*(epty-pty)/(epty-spty) #lng of intersection
    if xseg<ptx: #test point is on the right of segment
        return 0
    return 1  #test point intersects with segment

def WithinHex(ptx,pty,hexg): # 0 for no, 1 for yes
    #input test point and hexagon, all in format [lng, lat]
    
    if pty==hexg[2]:# test point is the lowest vertax
        if ptx==hexg[1]:
            return 1
        else:
            return 0
        
    if pty==hexg[8]:# test point is the highest vertax
        if ptx==hexg[7]:
            return 1
        else:
            return 0
        
    sc=0 # # of intersections
    for i in range(0,5):
        sc=sc+IntersectsSegment(ptx,pty,hexg[(2*i+1)],hexg[(2*i+2)],hexg[(2*i+3)],hexg[(2*i+4)])
    
    sc=sc+IntersectsSegment(ptx,pty,hexg[11],hexg[12],hexg[1],hexg[2])

    if sc%2==1:# if the # of intersections is odd
        return 1 # test point is in the hexagon
    else:# if the # of intersections is even
        return 0 # test point is not in the hexagon
    
def Get_grid(lng,lat,Grid_range):
    for i in range(len(Grid_range)):
        if WithinHex(lng,lat,Grid_range.iloc[i]):
            return Grid_range.iloc[i,0]
        else:
            pass


# Spatial Range

In [31]:
'''Grid Range'''

Grid_range=pd.read_csv('./Experiments/Demo/GridIndex.csv')

Grid_range=Grid_range.drop(columns=['Unnamed: 0','Index'])


# Data processing

In [33]:
date_str='2016-11-01'

GPS_path='Dataset/gps_'+date_str.replace('-','')+'.csv'

Order_path='Dataset//order_01-30/total_ride_request/order_'+date_str.replace('-','')+'.csv'

Tracectory_data=pd.read_csv(GPS_path,header=None,\
                            names = ['Driver_ID','Order_ID','Time_Stamp','Longitude','Latitude'])

Tracectory_data=Tracectory_data.iloc[:100]

Tracectory_data['Time_step']=Tracectory_data.apply(lambda x:stamp_to_step(x['Time_Stamp'],date_str,300), axis=1)

Tracectory_data=Tracectory_data.drop_duplicates(subset='Time_step', keep='last', inplace=False)

Tracectory_data['Grid']=Tracectory_data.apply(lambda x:Get_grid(x['Longitude'],x['Latitude'],Grid_range), axis=1)

Tracectory_data=Tracectory_data[['Driver_ID','Order_ID','Time_step','Grid']]

Tracectory_data

,Driver_ID,Order_ID,Time_step,Grid
16,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,131,889386c312e02f8a
88,8f20c9188561b796ef8e26196de30be4,11746cc1d82418066fda69f08fb671c8,181,83fd0dce987f46ea
99,8f20c9188561b796ef8e26196de30be4,11746cc1d82418066fda69f08fb671c8,182,83fd0dce987f46ea


In [39]:
Order_data=pd.read_csv(Order_path,header=None,\
                            names = ['Order_ID','Start_time','Stop_time','Pickup_Longitude','Pickup_Latitude',\
                                    'Dropoff_Longitude','Dropoff_Latitude','Reward_unit'])

Order_data=Order_data.iloc[:100]

Order_data['Start_step']=Order_data.apply(lambda x:stamp_to_step(x['Start_time'],date_str,300), axis=1)

Order_data['Stop_step']=Order_data.apply(lambda x:stamp_to_step(x['Stop_time'],date_str,300), axis=1)

Order_data['Pickp_Grid']=Order_data.apply(lambda x:Get_grid(x['Pickup_Longitude'],x['Pickup_Latitude'],Grid_range), axis=1)

Order_data['Dropoff_Grid']=Order_data.apply(lambda x:Get_grid(x['Dropoff_Longitude'],x['Dropoff_Latitude'],Grid_range), axis=1)

Order_data=Order_data[['Order_ID','Start_step','Stop_step','Pickp_Grid','Dropoff_Grid','Reward_unit']]

Order_data

,Order_ID,Start_step,Stop_step,Pickp_Grid,Dropoff_Grid,Reward_unit
0,eb9dd4095d9850e6287cefd813775a6c,117,123,9d2739e10948e232,541104d3641295d1,3.54
1,387a742fa5a3fbe4a1f215ac58ea33a8,186,193,e60c5c007507bcb0,7461298166b59abb,11.70
2,9cf55f8e6e02a1e0f792df06e5d85011,251,255,390a600ae7dad222,f2de7e1d0d4f0637,5.01
3,5feeae0307e15203484b9ffceef89855,200,204,1a58df7312309bd0,83fd0dce987f46ea,4.22
4,ad4b52cb15b90c44c8f20a8b1e57a622,94,96,ccbdef31d728b7a0,310c5bac13100430,2.27
5,ad551eb23b72e2a77ae5e81e22f2dbd0,226,230,9afea425734c0d96,e01ce17ed728a211,3.01
6,ad4b52cb15b90c44c8f20a8b1e57a622,94,96,ccbdef31d728b7a0,310c5bac13100430,2.27
7,db46d8931c1ac3f61d8278df67c2a365,97,101,5bc416b12315fec4,f58efa07b5f95620,4.97
8,908e7f068da5768c492ab69dbf81efda,103,107,28129de1881e438e,6c02d478a91414bc,4.90
9,2d48affae032a4ca29d220660732686d,108,113,6c02d478a91414bc,c8bcc56cd00c97c1,4.61
